In [ ]:
!pip install -U transformers

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data = data[['comment_text','toxic']]
data = data[0:1000]


In [ ]:
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , recall_score , precision_score , f1_score

import torch
from transformers import TrainingArguments , Trainer
from transformers import BertTokenizer , BertForSequenceClassification

In [ ]:
from transformers import BertTokenizer , BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to('cuda')


In [ ]:
sample_data = ["I am eating apple","I am playing cricket"]

tokenizer(sample_data, padding=True, truncation=True, max_length=512,return_tensors="pt")

{'input_ids': tensor([[ 101, 1045, 2572, 5983, 6207,  102],
        [ 101, 1045, 2572, 2652, 4533,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]])}

In [ ]:
X = list(data["comment_text"])
y = list(data["toxic"])

X_train , X_val , y_train ,y_val = train_test_split(X,y,test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      if self.labels:
        item["labels"] = torch.tensor(self.labels[idx])
      return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]

{'input_ids': tensor([  101,  2060,  5747,  4790,  2025, 14964,  1999,  2577,  1059,  1012,
          5747,  3904,  1997,  1996,  2206,  4790,  2003,  3855,  1999,  1996,
          2577,  1059,  1012,  5747,  1998,  3383,  2323,  2022,  1024,  5747,
          1005,  1055,  4167,  1024,  2129,  6382, 20996,  3726,  2081,  2577,
          1059,  1012,  5747,  4883,  4968,  3343,  1997,  1996,  2577,  1059,
          1012,  5747,  3447,  2220,  2166,  1997,  2577,  1059,  1012,  5747,
          3171,  3343,  1997,  1996,  2577,  1059,  1012,  5747,  3447,  6092,
          2381,  1997,  2577,  1059,  1012,  5747,  7214,  3550, 13954,  2015,
          1997,  2577,  1059,  1012,  5747,  3097,  3343,  1997,  1996,  2577,
          1059,  1012,  5747,  3447,  2577,  1059,  1012,  5747,  1998,  1996,
          5712,  2162,  2577,  1059,  1012,  5747,  2004,  3099,  1997,  3146,
          2577,  1059,  1012,  5747,  5239,  2577,  1059,  1012,  5747,  4883,
          3049,  2577,  1059,  1012,  5

In [ ]:
def compute_metrics(p):
  print(type(p))
  pred, labels = p
  pred = np.argmax(pred, axis=1)

  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred)
  precision = precision_score(y_true=labels, y_pred=pred)
  f1 = f1_score(y_true=labels, y_pred=pred)

  return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
#!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
#!pip install transformers[torch]

In [ ]:
#Define Trainer

args = TrainingArguments(
    output_dir = "output",
    num_train_epochs = 1,
    per_device_train_batch_size = 8,

)

In [ ]:

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.2973023796081543, metrics={'train_runtime': 73.9339, 'train_samples_per_second': 10.82, 'train_steps_per_second': 1.353, 'total_flos': 210488844288000.0, 'train_loss': 0.2973023796081543, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.19601337611675262,
 'eval_accuracy': 0.915,
 'eval_precision': 1.0,
 'eval_recall': 0.19047619047619047,
 'eval_f1': 0.32,
 'eval_runtime': 5.5546,
 'eval_samples_per_second': 36.006,
 'eval_steps_per_second': 4.501,
 'epoch': 1.0}

In [ ]:
text = "That biryani was tasty"

inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model(**inputs)

print(outputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

predictions = predictions.cpu().detach().numpy()
predictions



SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1792, -0.5927]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.6839, 0.3161]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.6839261, 0.3160739]], dtype=float32)

In [ ]:
trainer.save_model("CustomModel")

In [ ]:
#Load save model

model_2 = BertForSequenceClassification.from_pretrained("CustomModel")
model_2 = model_2.to("cuda")

In [ ]:
text = "he is a good man"
text = "go to hell"

inputs = tokenizer(text,padding=True, truncation=True, return_tensors="pt").to("cuda")
outputs = model_2(**inputs)

print(outputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3498, -0.1902]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.4602, 0.5398]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.46018153, 0.53981847]], dtype=float32)

In [ ]:
!pip install huggingface_hub

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification


In [ ]:
model = BertForSequenceClassification.from_pretrained('CustomModel')
model.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub('abdulqadir02/FineTunedBert')

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abdulqadir02/FineTunedBert/commit/9535fd766abec1f97ad07a70f74b231e1c8da9df', commit_message='Upload BertForSequenceClassification', commit_description='', oid='9535fd766abec1f97ad07a70f74b231e1c8da9df', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model=BertForSequenceClassification.from_pretrained('abdulqadir02/FineTunedBert')

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]